# Предобученная модель YOLO для детектинга людей  



In [6]:
from ultralytics import YOLO
import cv2

# Загрузка предобученной модели YOLOv8 (можно 'yolov8n.pt', 'yolov8s.pt' и т.д.)
model = YOLO('yolov8n.pt')  # nano-версия (быстрая, но менее точная)

In [13]:
# Загрузка изображения
#image = cv2.imread('2.png')
image = cv2.imread('E://AIM/AI-Flow-Detecting/ai-core/data/2025-08-08_11-29-04.png')

# Детекция людей (класс '0' в COCO = человек)
results = model(image, classes=[0])  

# Визуализация результатов
annotated_image = results[0].plot()  # Рисует bounding boxes
cv2.imshow('Detected People', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Подсчёт количества людей
people_count = len(results[0].boxes)
print(f"Количество людей на фото: {people_count}")


Количество людей на фото: 6


In [14]:
from ultralytics import YOLO
import cv2

# Загрузка модели YOLOv8 (nano — быстро, подходит для реального времени)
model = YOLO('yolov8n.pt')

# URL видеопотока (убраны лишние пробелы в конце!)
stream_url = "https://restreamer.vms.evo73.ru/1376e7dd864af96d/stream2.ts"

# Открываем видеопоток
cap = cv2.VideoCapture(stream_url)

if not cap.isOpened():
    print("❌ Ошибка: Не удалось открыть видеопоток. Проверь URL и соединение.")
    exit()

print("✅ Видеопоток запущен. Детекция людей через YOLOv8...")

while True:
    ret, frame = cap.read()

    if not ret:
        print("⚠️ Не удалось получить кадр. Поток может быть разорван.")
        break

    # Детекция ТОЛЬКО людей (класс 0 в COCO)
    results = model.predict(frame, classes=[0], conf=0.5, verbose=False)

    # Наносим bounding boxes и метки
    annotated_frame = results[0].plot()

    # Подсчёт обнаруженных людей
    people_count = len(results[0].boxes)
    cv2.putText(annotated_frame, f'Людей: {people_count}', (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2, cv2.LINE_AA)

    # Показываем кадр
    cv2.imshow('YOLOv8 — Детекция людей в реальном времени', annotated_frame)

    # Выход по клавише 'q' или пробелу
    if cv2.waitKey(1) & 0xFF in [ord('q'), ord(' ')]:
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()
print("⏹️ Работа завершена.")

✅ Видеопоток запущен. Детекция людей через YOLOv8...
⚠️ Не удалось получить кадр. Поток может быть разорван.
⏹️ Работа завершена.
